# 1.0 Carregamento dos Dados

**Objetivo:** Carregar o dataset de doenças cardiovasculares do Kaggle 

## Importação de Bibliotecas

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import mlcroissant as mlc
import logging

sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

# Suprimir warnings gerais
import warnings
warnings.filterwarnings('ignore')

# Suprimir warnings específicos do ML Croissant (ABSL)
logging.getLogger('absl').setLevel(logging.ERROR)

## Carregamento dos Dados

Devido a problemas de conexão com a API do Kaggle, optou-se pelo carregamento do dataset por meio do padrão ML Croissant, utilizando os metadados disponibilizados pela própria plataforma. Essa abordagem permitiu acessar os dados de forma estruturada e reprodutível, sem dependência direta da API.

In [7]:
# URL do dataset com metadados Croissant
dataset_url = 'https://www.kaggle.com/datasets/jocelyndumlao/cardiovascular-disease-dataset/croissant/download'

# Fetch the Croissant JSON-LD
print("Carregando dataset via ML Croissant...")
croissant_dataset = mlc.Dataset(dataset_url)

# Check what record sets are in the dataset
record_sets = croissant_dataset.metadata.record_sets
print(f"✓ {len(record_sets)} record set(s) encontrado(s)")
for rs in record_sets:
    print(f"  - {rs.name}: {rs.description}")

Carregando dataset via ML Croissant...
✓ 1 record set(s) encontrado(s)
  - Cardiovascular_Disease_Dataset.csv: Cardiovascular_Disease_Dataset


Além do carregamento, foi necessária a correção de tipos de dados, uma vez que a coluna patientid foi carregada em formato binário (bytes). Assim, os valores dessa coluna foram decodificados e convertidos para o tipo inteiro, garantindo a consistência do dataset para as etapas posteriores de processamento.

In [8]:
# Fetch the records and put them in a DataFrame
print("Carregando registros...")
records = list(croissant_dataset.records(record_set=record_sets[0].uuid))
df = pd.DataFrame(records)

# Limpar nomes das colunas (remover QUALQUER prefixo antes da barra)
# Exemplo: 'cardio_train.csv/age' -> 'age' ou 'Cardiovascular_Disease_Dataset.csv/age' -> 'age'
df.columns = [col.split('/')[-1] if '/' in col else col for col in df.columns]

# Corrigir coluna patientid (converter de bytes para int)
print("\nCorrigindo coluna patientid...")
df['patientid'] = df['patientid'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df['patientid'] = df['patientid'].astype(int)
print("✓ patientid convertido de bytes para int")


print(f"✓ Dados carregados: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
print(f"\nColunas disponíveis:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

Carregando registros...

Corrigindo coluna patientid...
✓ patientid convertido de bytes para int
✓ Dados carregados: 1,000 linhas × 14 colunas

Colunas disponíveis:
  1. patientid
  2. age
  3. gender
  4. chestpain
  5. restingBP
  6. serumcholestrol
  7. fastingbloodsugar
  8. restingrelectro
  9. maxheartrate
  10. exerciseangia
  11. oldpeak
  12. slope
  13. noofmajorvessels
  14. target


In [9]:
df.head()

,patientid,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
0,103368,53,1,2,171,0,0,1,147,0,5.3,3,3,1
1,119250,40,1,0,94,229,0,1,115,0,3.7,1,1,0
2,119372,49,1,2,133,142,0,0,202,1,5.0,1,0,0
3,132514,43,1,0,138,295,1,1,153,0,3.2,2,2,1
4,146211,31,1,1,199,0,0,2,136,0,5.3,3,2,1


### Salvando os dados

In [10]:
# Salvar dataset localmente em data/raw
raw_data_dir = Path('../data/raw')
raw_data_dir.mkdir(parents=True, exist_ok=True)
csv_file = 'Cardiovascular_Disease_Dataset.csv'
dest_file = raw_data_dir / csv_file

df.to_csv(dest_file, index=False)
print(f"✓ Dataset salvo em: {dest_file}")
print(f"  Tamanho do arquivo: {dest_file.stat().st_size / 1024:.2f} KB")

✓ Dataset salvo em: ..\data\raw\Cardiovascular_Disease_Dataset.csv
  Tamanho do arquivo: 42.78 KB
